In [1]:
import pickle
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from joblib import dump, load
from cartopy.util import add_cyclic_point

import warnings
warnings.filterwarnings('ignore')

import matplotlib.path as mpath
import cartopy
import cartopy.crs as ccrs
from matplotlib.gridspec import GridSpec
import geopandas as gpd
crs_sp = ccrs.SouthPolarStereo(central_longitude=0)
crs_ll = ccrs.PlateCarree()

## Load data

In [5]:
#ERA5 data
era = xr.open_dataset('/glade/work/devond/ERA5/era_1985_2015.nc')
era_precip = era.pr
era_ws_annual = era.wind
era_ta_annual = era.ta
era_ta_summer = era.ta_s

In [19]:
def run_emulator(precip, ws, ta_a, ta_s, m):
    #prepare data for emulator
    pr = precip.rolling(year=10, center=True).mean(dim='year').dropna("year")
    ws = ws.rolling(year=10, center=True).mean(dim='year').dropna("year")
    ta_a = ta_a.rolling(year=10, center=True).mean(dim='year').dropna("year")
    ta_s = ta_s.rolling(year=10, center=True).mean(dim='year').dropna("year")

    precip = pr.mean(dim = 'year').values
    wind = ws.mean(dim = 'year').values
    temp = ta_a.mean(dim = 'year').values
    temp_summer = ta_s.mean(dim = 'year').values
    
    X = np.vstack([precip.flatten(), wind.flatten(), temp.flatten(), temp_summer.flatten()]).T
    X = (X-means)/stds
    
    #run emulator
    fac = regr.predict(X)
    fac_reshape = fac.reshape(precip.shape)
    
    lon = pr.lon.values
    lat = pr.lat.values
        
    #create dataset
    current = xr.Dataset(
        data_vars=dict(
            pr=(["lat", "lon"], precip),
            wind=(["lat", "lon"], wind),
            ta=(["lat", "lon"], temp),
            ta_s=(["lat", "lon"], temp_summer),
            fac=(["lat", "lon"], fac_reshape),
        ),
        coords=dict(
            lon=lon,
            lat=lat,
        ),
    )
    
    return current

def load_model(run):
    with open(f'/glade/work/devond/FAC/emulators/weights_{run}.p','rb') as fp: #load means and standard deviations to standardize data for random forest prediction
        weights = pickle.load(fp)
        means = weights[0]
        stds = weights[1]
    
    return means, stds
    

In [27]:
run = 'TP1'
regr = load(f'emulators/random_forest_{run}.sav') #load in random forest regressor
means, stds = load_model(run)

era_ds = run_emulator(era_precip, era_ws_annual, era_ta_annual, era_ta_summer, 'ERA')
era_ds.to_netcdf(f'netcdf_results/emulator_FAC/era_{run}.nc')